<a href="https://colab.research.google.com/github/esthy13/cil-intrusion-detection/blob/main/der_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DER++ for Intrusion Detection (CIC-IDS)

Minimal **working** implementation of **Dark Experience Replay++**
for class-incremental intrusion detection.

In [51]:
!git clone https://github.com/esthy13/cil-intrusion-detection
%cd cil-intrusion-detection
!git pull
# resetting the path to content to avoid issues in the rest of the notebook
%cd ..

fatal: destination path 'cil-intrusion-detection' already exists and is not an empty directory.


/content/cil-intrusion-detection
Already up to date.
/content


In [52]:
import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataset

In [53]:
class IDSBaseDataset(Dataset):
    def __init__(self, root_dir, split="train"):
        """
        root_dir: path to 2017/
        split: 'train' or 'test'
        """
        csv_dir = os.path.join(root_dir, split)
        csvs = glob.glob(os.path.join(csv_dir, "*.csv"))
        assert len(csvs) > 0, f"No CSV files found in {csv_dir}"

        df = pd.concat([pd.read_csv(c) for c in csvs], ignore_index=True)

        labels = list(df["Label"].unique())

        if "benign" not in labels:
            raise ValueError("Dataset must contain a 'benign' class")

        # Enforcing benign as class 0
        labels = ["benign"] + sorted([l for l in labels if l != "benign"])

        self.classes = labels
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

        self.x = df.drop(columns=["Label"]).values.astype(np.float32)
        self.y = np.array(
            [self.class_to_idx[label] for label in df["Label"]],
            dtype=np.int64
        )

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])

## Task builder

In [54]:
class RemappedSubset(Dataset):
    """
    Subset that remaps global class indices to [0..C-1]
    """
    def __init__(self, dataset, indices, class_ids):
        self.dataset = dataset
        self.indices = indices
        self.class_map = {cid: i for i, cid in enumerate(class_ids)}

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        x, y = self.dataset[self.indices[idx]]
        return x, torch.tensor(self.class_map[y.item()])


In [55]:
def build_task(dataset, class_names):
    class_ids = [dataset.class_to_idx[c] for c in class_names]
    idxs = np.where(np.isin(dataset.y, class_ids))[0]
    return RemappedSubset(dataset, idxs, class_ids)

In [56]:
def build_scenario( all_classes, attacks_pattern, benign_class="benign"):
    """
    all_classes: ordered list of class names (benign must be first)
    attacks_pattern: list of ints, number of NEW attacks per task
                     e.g. [1,1,1] or [3,2] or [5]
    benign_class: name of benign class (default: 'benign')

    Returns:
        tasks: list of lists of class names (cumulative)
    """

    if benign_class not in all_classes:
        raise ValueError(f"Benign class '{benign_class}' not found in classes")

    if all_classes[0] != benign_class:
        raise ValueError(
            f"Benign class must be index 0, got {all_classes[0]}"
        )

    attack_classes = [c for c in all_classes if c != benign_class]

    if sum(attacks_pattern) != len(attack_classes):
        raise ValueError(
            f"Invalid attacks_pattern: sum={sum(attacks_pattern)}, "
            f"but there are {len(attack_classes)} attack classes"
        )

    tasks = []
    current_index = 0

    for _, n_new in enumerate(attacks_pattern):
        current_index += n_new
        seen_attacks = attack_classes[:current_index]
        seen_classes = [benign_class] + seen_attacks
        tasks.append(seen_classes)

    return tasks


## Model

In [57]:
class CILModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fe = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU()
        )
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        z = self.fe(x)
        return self.classifier(z)

    def expand_classes(self, n_new):
        old = self.classifier
        new = nn.Linear(old.in_features, old.out_features + n_new).to(device)
        new.weight.data[:old.out_features] = old.weight.data
        new.bias.data[:old.out_features] = old.bias.data
        self.classifier = new

## Replay Buffer (DER++)

In [58]:
class ReservoirBuffer:
    def __init__(self, size):
        self.size = size
        self.n_seen = 0
        self.x, self.y, self.logits = [], [], []

    def add(self, x, y, logits):
        for xi, yi, li in zip(x, y, logits):
            li = li.detach().cpu()
            if len(self.x) < self.size:
                self.x.append(xi.cpu())
                self.y.append(yi.cpu())
                self.logits.append(li)
            else:
                j = np.random.randint(0, self.n_seen + 1)
                if j < self.size:
                    self.x[j] = xi.cpu()
                    self.y[j] = yi.cpu()
                    self.logits[j] = li
            self.n_seen += 1

    def sample(self, batch_size, current_dim):
        if len(self.x) == 0:
            return None

        idx = np.random.choice(len(self.x), min(batch_size, len(self.x)), replace=False)

        bx = torch.stack([self.x[i] for i in idx]).to(device)
        by = torch.stack([self.y[i] for i in idx]).to(device)

        # PAD LOGITS
        padded_logits = []
        for i in idx:
            old_logit = self.logits[i]
            if old_logit.numel() < current_dim:
                pad = torch.zeros(current_dim - old_logit.numel())
                old_logit = torch.cat([old_logit, pad])
            padded_logits.append(old_logit)

        blogits = torch.stack(padded_logits).to(device)

        return bx, by, blogits

## DER++ Training Loop

In [59]:
def train_task(model, loader, buffer, optimizer, alpha=0.5, beta=0.5, epochs=1):
    ce = nn.CrossEntropyLoss()

    model.train()
    for _ in range(epochs):
        for x, y in loader:
            x, y = x.to(device), y.to(device)

            out = model(x)
            loss = ce(out, y)

            buf = buffer.sample(len(x), model.classifier.out_features)
            if buf:
                bx, by, blog = buf
                loss += alpha * F.mse_loss(model(bx), blog)
                loss += beta * ce(model(bx), by)



            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            buffer.add(x, y, out.detach())

In [60]:
def evaluate(model, dataset, seen_classes):
    model.eval()

    eval_dataset = build_task(dataset, seen_classes)
    loader = DataLoader(eval_dataset, batch_size=256, shuffle=False)

    all_preds, all_targets = [], []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            preds = model(x).argmax(1).cpu().numpy()

            all_preds.append(preds)
            all_targets.append(y.numpy())

    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    acc = accuracy_score(all_targets, all_preds)
    f1  = f1_score(all_targets, all_preds, average="macro")

    return acc, f1


## Run Experiment

In [61]:
# !unzip cil-intrusion-detection/data/processed/2017.zip

In [62]:
# Paths
DATA_ROOT = "2017"  # <-- folder created by unzip

# Datasets
train_dataset = IDSBaseDataset(DATA_ROOT, split="train")
test_dataset  = IDSBaseDataset(DATA_ROOT, split="test")

print(train_dataset.class_to_idx)


{'benign': 0, 'bot': 1, 'ddos': 2, 'dos': 3, 'ftp-patator': 4, 'portscan': 5, 'ssh-patator': 6, 'web-attack': 7}


In [65]:
input_dim = train_dataset.x.shape[1]

# Task definition (example)
all_classes = [
    "benign",
    "dos",
    "ddos",
    "portscan",
    "ssh-patator",
    "ftp-patator",
    "web-attack",
    "bot"
]

# Scenario A: 1+1+1+1+1+1+1+1
scenario_1 = build_scenario(all_classes, [1,1,1,1,1,1,1])

# Scenario B: 5+3
scenario_2 = build_scenario(all_classes, [4, 3])

# Scenario C: 2+3+3
scenario_3 = build_scenario(all_classes, [1, 3, 3])

for scenario_id, tasks in enumerate([scenario_1, scenario_2, scenario_3]):
    print(f"\n=== Scenario {scenario_id+1} ===")
    model = CILModel(input_dim, len(tasks[0])).to(device)
    buffer = ReservoirBuffer(size=2000)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for task_id, seen_classes in enumerate(tasks):
        print(f"\n=== Task {task_id}: {seen_classes}")

        if task_id > 0:
            n_new = len(tasks[task_id]) - len(tasks[task_id-1])
            model.expand_classes(n_new)
            optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

        train_loader = DataLoader(
            build_task(train_dataset, seen_classes),
            batch_size=128,
            shuffle=True
        )

        train_task(model, train_loader, buffer, optimizer)

        acc, f1 = evaluate(model, test_dataset, seen_classes)
        print(f"Accuracy: {acc:.4f} | Macro-F1: {f1:.4f}")



=== Scenario 1 ===

=== Task 0: ['benign', 'dos']
Accuracy: 0.7781 | Macro-F1: 0.4563

=== Task 1: ['benign', 'dos', 'ddos']
Accuracy: 0.8287 | Macro-F1: 0.3136

=== Task 2: ['benign', 'dos', 'ddos', 'portscan']
Accuracy: 0.7812 | Macro-F1: 0.2291

=== Task 3: ['benign', 'dos', 'ddos', 'portscan', 'ssh-patator']
Accuracy: 0.7856 | Macro-F1: 0.2568

=== Task 4: ['benign', 'dos', 'ddos', 'portscan', 'ssh-patator', 'ftp-patator']
Accuracy: 0.7791 | Macro-F1: 0.2146

=== Task 5: ['benign', 'dos', 'ddos', 'portscan', 'ssh-patator', 'ftp-patator', 'web-attack']
Accuracy: 0.7764 | Macro-F1: 0.1295

=== Task 6: ['benign', 'dos', 'ddos', 'portscan', 'ssh-patator', 'ftp-patator', 'web-attack', 'bot']
Accuracy: 0.7806 | Macro-F1: 0.1148

=== Scenario 2 ===

=== Task 0: ['benign', 'dos', 'ddos', 'portscan', 'ssh-patator']
Accuracy: 0.1934 | Macro-F1: 0.1179

=== Task 1: ['benign', 'dos', 'ddos', 'portscan', 'ssh-patator', 'ftp-patator', 'web-attack', 'bot']
Accuracy: 0.1950 | Macro-F1: 0.0695

==

**Scenario 1** does not perform well the model fails to learn the new classes
**Scenario 2** and **Scenario 3** macro f1 score gets better with more classes which means that the model is working better